In [ ]:
!pip install numpy networkx overpy 

In [ ]:
import pickle
import networkx as nx
import numpy as np
import overpy
import matplotlib.pyplot as plt

In [ ]:
fetch_data = True

In [ ]:
from ast import NotEq
def _gcdist(lat0:np.single, lon0:np.single, lat1:np.single, lon1:np.single)->np.single:
    """
    Calculate the great circle distance between two points in meters using a spherical model of the earth (good for distances less than 475km)
    :param lat0:
    :param lon0:
    :param lat1:
    :param lon1:
    :return:
    """
    delta_lat = np.radians(lat1 - lat0)
    delta_lon = np.radians(lon1 - lon0)
    a = np.sin(delta_lat/2)**2+np.cos(np.radians(lat0))*np.cos(np.radians(lat1))*np.sin(delta_lon/2)**2
    c = 2*np.arcsin(np.sqrt(a))
    return c*np.single(6371000)

def __weight(way:overpy.Way, node1:int=None, node2:int=None):
    if node is not None and node2 is not None:
      i1 = way.nodes.index(node1)
      i2 = way.nodes.index(node2)
      if i1>i2:
        i1,i2 = i2,i1
    else:
      n1 =0
      n2=len(way.nodes)
    weight =0
    for i in range(n1,n2-1):
        weight+=_gcdist(float(way.nodes[i].lat), float(way.nodes[i].lon), float(way.nodes[i+1].lat), float(way.nodes[i+1].lon))
    return weight

In [ ]:
if fetch_data:
    api = overpy.Overpass()
    umb_data = api.query(
        'way["highway"~"pedestrian|footway|steps|sidewalk|cycleway|path|corridor"](poly: "42.3196561 -71.0520413 42.3202193 -71.0510167 42.3175209 -71.0415681 42.3134307 -71.0430135 42.3099378 -71.0373486 42.3133297 -71.0324853 42.3189148 -71.0332685 42.3235297 -71.0451844 42.3212511 -71.0521287 42.3196561 -71.0520413");(._;>;);out;'
    )
    print(umb_data)
    with open("umb_data.pkl", "wb") as f:
        pickle.dump(umb_data, f)
else:
    with open("umb_data.pkl", "rb") as f:
        umb_data = pickle.load(f)
nodes_count = np.zeros((len(umb_data.nodes),5), dtype=np.int64)
nodes_count[:,0]=np.array(umb_data.node_ids)
for way in umb_data.ways:
    if way.id ==1018144900:
        print(f"{way.id} = {way.nodes}")
    for node in way.nodes:
        if node.id == 12660490877:
            print(node)
        if "entrance" in node.tags.keys():
            if node.tags["entrance"] == "yes":
                nodes_count[np.where(nodes_count == node.id)[0][0], 1] +=2
        nodes_count[np.where(nodes_count == node.id)[0][0], 1] += 1
graph_nodes = nodes_count[nodes_count[:,1]>1][:,0]

In [ ]:
graph = nx.Graph()
graph.add_nodes_from(graph_nodes)
for way in umb_data.ways:
    # if way.id == 1018144900 or way.id== 1019459515:
        # print(f"{way.id} points:{way._node_ids}")
    if "area" in way.tags.keys() and way.tags["area"]=="yes":
      nodes = np.array(way._node_ids,dtype=np.int64)
    else:
      nodes = np.intersect1d(graph_nodes, way._node_ids)
    for i in range (len(nodes)-1):
            graph.add_edge(nodes[i], nodes[i+1], weight=__weight(way))
            # print(f"added edge between {nodes[i]} and { nodes[j]}")

In [ ]:
# graph.remove_nodes_from(list(nx.isolates(graph)))
graph.number_of_edges()

In [ ]:
path = nx.astar_path(graph, 327832796, 12660490877)
path

In [ ]:
# nx.draw(graph,with_labels=True)

In [ ]:
nx.draw_spring(graph,with_labels=True)

In [ ]:
nx.number_connected_components(graph)

In [ ]:
components_set = list(nx.connected_components(graph))
for components in components_set:
  print(len(components))
  if len(components)<10:
    graph.remove_nodes_from(components)
print(components_set[1])